In [1]:
%load_ext autoreload

%autoreload 2
import numpy as np
import dynet as dy
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline

from infer import WrappedTagger
from utils import Instance

import seaborn as sns
sns.set()

### Datasets/Models

In [2]:
model = WrappedTagger('datasets/ud-2.3/en_ud23.pkl',
                      num_lstm_layers=2,
                      hidden_dim=128,
                      char_embedding_dim=256,
                      word_level_dim=128)
model.load_weights('models/en/best_model_epoch-32-0.9365.bin')

Using legacy loading


### Available Alphabet 

In [3]:
print(' '.join(sorted(model.dataset.c2i.keys())))

! " # $ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < <*> = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~ £ ­ ³ ´ · Ã à á ç é Υ – — ‘ ’ “ ” … ♥


In [4]:
def show(sentence, dim=0):
    f, a = plt.subplots(1, figsize=(8, 8))
    out_tags, embeddings = model.forward_text(sentence)
    #this is the word level prediction!
    print(model.get_pos(out_tags))
    activations = []
    
    padded_ascii = []
    for word, word_embedding in zip(sentence.split(), embeddings):
        for char, char_embedding in zip('*' + word + '*', word_embedding):
            if char == '*':
                activations.append(np.nan)
                padded_ascii.append(' ')
            else:
                activations.append(char_embedding.npvalue()[dim])
                padded_ascii.append(char)
    a.plot(activations)
    a.set_xticks(range(len(activations)))
    a.set_xticklabels(padded_ascii)
    a.xaxis.set_ticks_position('none')

In [5]:
# #expects space tokenized inputs
# show('There are two wugs .', dim=0);

### Interactive 

In [6]:
word_input = widgets.Text(value='Now there are two of them .', continuous_update=False)
dim_input = widgets.IntSlider(min = 0, max = 127, step = 1, value = 0, continuous_update=False)
plot = interactive(show, sentence=word_input, dim=dim_input)
output = plot.children[-1]
output.layout.height = '600px'
plot

interactive(children=(Text(value='Now there are two of them .', continuous_update=False, description='sentence…